In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import time
from key import id, password

prefs = {
        "download.default_directory": "C:\\Users\\MIRAI\\programs\\syllabus\\pdf\\",
        "download.prompt_for_download": False,
        "plugins.always_open_pdf_externally": True,
        }

ID = id
PASS = password

ImportError: cannot import name 'id' from 'key' (c:\programs\PBL_stock\key.py)

In [8]:
options = Options()

options.add_experimental_option("detach", True)

options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)

driver.get("https://webcit-5.edu.metro-cit.ac.jp/users/sign_in")

driver.find_element(By.NAME, "user[name]").send_keys(ID)
driver.find_element(By.NAME, "user[password]").send_keys(PASS)

driver.find_element(By.NAME, "commit").click()

In [5]:
time.sleep(1)

driver.execute_script("window.open('https://webcit-5.edu.metro-cit.ac.jp/gnumbers/33494/last_hyoukas');")

# 現在のウィンドウハンドルを取得
original_window = driver.current_window_handle

# 新しいタブまたはウィンドウに切り替え
new_window = [window for window in driver.window_handles if window != original_window][0]
driver.switch_to.window(new_window)

In [6]:
rows = driver.find_elements(By.CSS_SELECTOR, "table.bordered tr")

for row in rows:
    cells = row.find_elements(By.CSS_SELECTOR, "td")
    if cells:
        last_cell = cells[-1]
        if (last_cell.text != "PDFファイルが作成させていません。"):
            # print(last_cell.text)
            last_cell.click()

In [36]:
import os
from PyPDF2 import PdfReader, PdfWriter

def merge_pdfs(directory, output_filename):
    # PdfWriter オブジェクトの初期化
    writer = PdfWriter()

    # 指定されたディレクトリ内のファイルをループする
    for item in sorted(os.listdir(directory)):
        if item.endswith('.pdf'):
            # PDFファイルを開く
            filepath = os.path.join(directory, item)
            reader = PdfReader(filepath)

            # 各ページを書き込む
            for page in reader.pages:
                writer.add_page(page)

    # 結合したPDFを保存
    with open(output_filename, 'wb') as f:
        writer.write(f)

# 使用例
directory = './pdf'
output_filename = './output.pdf'
merge_pdfs(directory, output_filename)

# sbiにあるか確認

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import json
import requests
from bs4 import BeautifulSoup
from key import sbi_id, sbi_pass

def remove_symbol_from_json(symbol):
    with open('./scrape_data/stock_name.json', 'r', encoding="utf-8") as file:
        data = json.load(file)

    for category in data:
        data[category] = [company for company in data[category] if company["symbol"] != symbol]

    with open('./scrape_data/stock_name.json', 'w', encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

driver.get("https://www.sbisec.co.jp/ETGate")

id_input = driver.find_element(By.NAME, "user_id")
id_input.send_keys(sbi_id)

pass_input = driver.find_element(By.NAME, "user_password")
pass_input.send_keys(sbi_pass)

driver.find_element(By.NAME, "ACT_login").click()

print("Please input Enter")
input("")

driver.get("https://global.sbisec.co.jp/home")
time.sleep(0.3)

driver.find_element(By.XPATH, r'//*[@id="root"]/main/article/div[1]/div[2]/a[2]').click()

# JSONファイルの読み込み
with open('./scrape_data/stock_name.json', 'r', encoding="utf-8") as file:
    data = json.load(file)

# 全てのシンボルをプリント
for category in data:
    for company in data[category]:
        stock_code = company["symbol"]
        stock_name = company["company_name"]
        print(stock_code, ':', stock_name)

        try:
            driver.get(f"https://global.sbisec.co.jp/invest/us/stock/{stock_code}?resource=news&searchType=include")
            time.sleep(0.3)

            iframe = driver.page_source
            soup = BeautifulSoup(iframe, "html.parser")
            url_pattern = "https://graph.sbisec.co.jp/sbinews/pc?"
            iframe_urls = [iframe.get('src') for iframe in soup.find_all('iframe', src=True) if iframe['src'].startswith(url_pattern)]

            token = iframe_urls[0].split("token=")[1]
            url_head = f"https://graph.sbisec.co.jp/sbinews/srvdetail?symbol={stock_code}&token={token}"
            print(url_head)

            res_head = requests.get(url_head)
            data_head = res_head.json()

            len_head = len(data_head['data'])
            time.sleep(1)

        except Exception as e:
            print(f"Error processing {stock_code}: {e}")
            remove_symbol_from_json(stock_code)
            print(f"Removed {stock_code} from stock_name.json")


Please input Enter
A : アジレント・テクノロジー Agilent Technologies, Inc.
https://graph.sbisec.co.jp/sbinews/srvdetail?symbol=A&token=2B190B71960D701E96D35A6ECDAD7E72B59CFE5C62EC8A4E23FF0259298BB46EB40695F80C6460B3123CD4FB7FCCF0E7BDD0D21BABAB8088A49AB6CCFC7AACEB132F67F8EECD5EB6BCEA2C51CD828C00
AAL : アメリカン・エアラインズ・グループ American Airlines Group Inc.
https://graph.sbisec.co.jp/sbinews/srvdetail?symbol=AAL&token=2B190B71960D701E96D35A6ECDAD7E72B59CFE5C62EC8A4E23FF0259298BB46EB40695F80C6460B3123CD4FB7FCCF0E7BDD0D21BABAB8088A49AB6CCFC7AACEB132F67F8EECD5EB6BCEA2C51CD828C00
AAPL : アップル Apple Inc.
https://graph.sbisec.co.jp/sbinews/srvdetail?symbol=AAPL&token=2B190B71960D701E96D35A6ECDAD7E72B59CFE5C62EC8A4E23FF0259298BB46EB40695F80C6460B3123CD4FB7FCCF0E7BDD0D21BABAB8088A49AB6CCFC7AACEB132F67F8EECD5EB6BCEA2C51CD828C00
ABBV : アッヴィ AbbVie Inc.
https://graph.sbisec.co.jp/sbinews/srvdetail?symbol=ABBV&token=2B190B71960D701E96D35A6ECDAD7E72B59CFE5C62EC8A4E23FF0259298BB46EB40695F80C6460B3123CD4FB7FCCF0E7BDD0D21BAB

In [5]:
import requests
import json

# stockData = json.load(open("./scrape_data/stock_name.json", encoding="cp932"))

with open('./stock_name.json', 'r', encoding="utf-8") as file:
    stockData = json.load(file)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100",
    "Accept-Language": "ja,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
}

for category in stockData:
    for company in stockData[category]:
        stock_code = company["symbol"]
        response = requests.get(
            f"https://api.nasdaq.com/api/quote/{stock_code}/info?assetclass=stocks",
            headers=headers,
        )
        fetchedData = response.json()
        if fetchedData["data"] == None:
            print("アッ！！", stock_code)
            # break

        print("ok", stock_code)

In [13]:
url = 'https://api.nasdaq.com/api/quote/NVDA/historical?assetclass=stocks&fromdate=2024-01-01&limit=9999&todate=2024-07-27&random=43'
res = requests.get(url, headers=headers)

data = res.json()
print(data['data']['tradesTable']['rows'])

date_close_list = [{"date": row["date"], "close": row["close"]} for row in data["data"]["tradesTable"]["rows"]]

dates = [row["date"] for row in data["data"]["tradesTable"]["rows"]]
closes = [row["close"] for row in data["data"]["tradesTable"]["rows"]]

print(date_close_list[0]['date'])
print("Dates:", dates)
print("Closes:", closes)

[{'date': '07/26/2024', 'close': '$113.06', 'volume': '293,399,100', 'open': '$116.19', 'high': '$116.20', 'low': '$111.58'}, {'date': '07/25/2024', 'close': '$112.28', 'volume': '460,067,000', 'open': '$113.04', 'high': '$116.63', 'low': '$106.30'}, {'date': '07/24/2024', 'close': '$114.25', 'volume': '327,776,900', 'open': '$119.17', 'high': '$119.95', 'low': '$113.44'}, {'date': '07/23/2024', 'close': '$122.59', 'volume': '173,911,000', 'open': '$122.78', 'high': '$124.69', 'low': '$122.10'}, {'date': '07/22/2024', 'close': '$123.54', 'volume': '258,068,900', 'open': '$120.35', 'high': '$124.07', 'low': '$119.86'}, {'date': '07/19/2024', 'close': '$117.93', 'volume': '217,223,800', 'open': '$120.35', 'high': '$121.60', 'low': '$117.37'}, {'date': '07/18/2024', 'close': '$121.09', 'volume': '320,979,500', 'open': '$121.85', 'high': '$122.40', 'low': '$116.56'}, {'date': '07/17/2024', 'close': '$117.99', 'volume': '390,086,200', 'open': '$121.35', 'high': '$121.85', 'low': '$116.72'},

In [6]:
import requests
import json
import pandas as pd

BASE_URL = 'http://192.168.1.222:8999'

with open('./stock_name.json', 'r', encoding='utf-8') as file:
    stockData = json.load(file)

for category in stockData:
    for company in stockData[category]:
        stock_code = company['symbol']
        url = f'{BASE_URL}/getdays/{stock_code}'
        res = requests.get(url)
        print(res)
        df = pd.DataFrame(res.json())
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
        date_time_array = df['datetime'].to_numpy()
        print(df.info())
        # print(date_time_array)

<Response [200]>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      2999 non-null   object        
 1   time      2999 non-null   object        
 2   datetime  2999 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 70.4+ KB
None
['2024-07-16T01:51:00.000000000' '2024-07-16T01:41:00.000000000'
 '2024-07-16T01:39:00.000000000' ... '2024-07-08T14:12:00.000000000'
 '2024-07-08T14:06:00.000000000' '2024-07-08T13:41:00.000000000']
<Response [200]>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      322 non-null    object        
 1   time      322 non-null    object        
 2   datetime  322 non-null    datetime64[ns]
dtypes: datetime64[ns](1), obje

KeyboardInterrupt: 

# values get

In [1]:
import requests
import json
from datetime import datetime

stocks_data = {}
today = datetime.today().strftime('%Y-%m-%d')

with open('./stock_name.json', 'r', encoding="utf-8") as file:
    stockData = json.load(file)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100",
    "Accept-Language": "ja,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
}

for category in stockData:
    for company in stockData[category]:
        stock_code = company["symbol"]

        url = f'https://api.nasdaq.com/api/quote/{stock_code}/historical?assetclass=stocks&fromdate=2024-01-01&limit=9999&todate={today}'

        response = requests.get(url, headers=headers)

        print(stock_code)
        print(response)

        if response.status_code == 200:
            # レスポンスから必要なデータを抽出（例：response.json()から 'rows' 部分を抽出）
            stock_data = response.json()["data"]["tradesTable"]["rows"]
            # 必要なフィールドのみ抽出して辞書に追加
            stocks_data[stock_code] = [{"date": row["date"], "close": row["close"]} for row in stock_data]
        else:
            print(f"Failed to retrieve data for {stock_code}")

with open('./stocks_values.json', 'w') as json_file:
    json.dump(stocks_data, json_file, indent=4)

print("JSONファイルに保存されました。")

A
<Response [200]>
AAL
<Response [200]>
AAPL
<Response [200]>
ABBV
<Response [200]>
ABNB
<Response [200]>
ABT
<Response [200]>
ACGL
<Response [200]>
ACN
<Response [200]>
ADBE
<Response [200]>
ADI
<Response [200]>
ADM
<Response [200]>
ADP
<Response [200]>
ADSK
<Response [200]>
AEE
<Response [200]>
AEP
<Response [200]>
AES
<Response [200]>
AFL
<Response [200]>
AIG
<Response [200]>
AIZ
<Response [200]>
AJG
<Response [200]>
AKAM
<Response [200]>
ALB
<Response [200]>
ALGN
<Response [200]>
ALL
<Response [200]>
ALLE
<Response [200]>
AMAT
<Response [200]>
AMCR
<Response [200]>
AMD
<Response [200]>
AME
<Response [200]>
AMGN
<Response [200]>
AMP
<Response [200]>
AMT
<Response [200]>
AMZN
<Response [200]>
ANET
<Response [200]>
ANSS
<Response [200]>
AON
<Response [200]>
AOS
<Response [200]>
APA
<Response [200]>
APD
<Response [200]>
APH
<Response [200]>
APTV
<Response [200]>
ARE
<Response [200]>
ATO
<Response [200]>
AVB
<Response [200]>
AVGO
<Response [200]>
AVY
<Response [200]>
AWK
<Response [200]

TypeError: 'NoneType' object is not subscriptable